In [1]:
import json
import os.path as osp
import platform
import time
import re
import csv
import random
import traceback
import logging
import pandas as pd

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.proxy import Proxy, ProxyType
from selenium.webdriver.common.action_chains import ActionChains

In [13]:
mai_dam = pd.read_csv("mại dâm.csv")
tong_hop = pd.read_csv("tổng hợp.csv")

In [18]:
mai_dam = mai_dam[['url', 'name', 'reply_num', 'view_num']]
tong_hop = tong_hop[['url', 'name', 'reply_num', 'view_num']]

In [24]:
#suffle
mai_dam = mai_dam.sample(frac=1)
tong_hop = tong_hop.sample(frac=1)

In [25]:
mai_dam

,url,name,reply_num,view_num
559,xamvn.vc/r/69-voi-em-em-pga-cao-cap-ngon-vkl.6...,69 với em Em PGA cao cấp ngon vkl ...,76.0,20000.0
373,xamvn.vc/r/poker-xi-to-mon-the-thao-tri-tue.14...,Poker/xì tố - môn thể thao trí tuệ,51.0,3000.0
623,xamvn.vc/r/tiktok-tay-co-khac.654588/,Tiktok Tây có khác,12.0,4000.0
185,xamvn.vc/r/em-nhung-massage-az-van-quan.639178/,Em nhung massage az văn quán,18.0,3000.0
274,xamvn.vc/r/ai-di-em-2k-nay-roi-review-nhi.649953/,Ai đi em 2k này rồi review nhỉ,35.0,5000.0
...,...,...,...,...
722,xamvn.vc/r/nho-cao-nhan-unlock-ig-private-dund...,Nhờ cao nhân unlock ig private dundun.meomeo,19.0,3000.0
414,xamvn.vc/r/cho-may-thang-chua-biet-molly-voi-m...,Cho mấy thằng chưa biết molly với mdma nguyên ...,734.0,40000.0
746,xamvn.vc/r/tim-clip-con-di-nho-vu-to-da-trang-...,Tìm clip Con đĩ nhỏ vú to da trắng dịu dàng si...,69.0,21000.0
17,xamvn.vc/r/xin-phep-mod-giai-toa-noi-buc-xuc-m...,Xin phép mod giải tỏa nỗi bức xúc ms lành mạnh,16.0,557.0


In [16]:
class Collect_Xamvn_Info:
    def __init__(self, no_gui=False, proxy=None):
        
        self.result = []
        executable = ''
        
        #LƯU Ý
        # tự chỉnh đường dẫn theo máy 
        if platform.system() == 'Windows':
            print('Detected OS : Windows')
            executable = 'chromedriver.exe'
        elif platform.system() == 'Linux':
            print('Detected OS : Linux')
            executable = './chromedriver/chromedriver_linux'
        elif platform.system() == 'Darwin':
            print('Detected OS : Mac')
            executable = './chromedriver/chromedriver_mac'
        else:
            raise OSError('Unknown OS Type')

        if not osp.exists(executable):
            raise FileNotFoundError(
                'Chromedriver file should be placed at {}'.format(executable))

        chrome_options = Options()
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')
        chrome_options.add_argument('ignore-certificate-errors')
        chrome_options.add_argument('--ignore-ssl-errors=yes')

        if no_gui:
            chrome_options.add_argument('--headless')
        if proxy:
            chrome_options.add_argument("--proxy-server={}".format(proxy))

        self.browser = webdriver.Chrome(executable_path=executable, chrome_options=chrome_options)    

        #auto download driver nhưng mà web chính thức chưa có chrome driver mới nhất version 116 nên phải làm bằng cơm :D
        
#         self.browser = webdriver.Chrome(
#             ChromeDriverManager().install(), chrome_options=chrome_options)

#         browser_version = 'Failed to detect version'
#         chromedriver_version = 'Failed to detect version'
#         major_version_different = False

#         if 'browserVersion' in self.browser.capabilities:
#             browser_version = str(self.browser.capabilities['browserVersion'])

#         if 'chrome' in self.browser.capabilities:
#             if 'chromedriverVersion' in self.browser.capabilities['chrome']:
#                 chromedriver_version = str(
#                     self.browser.capabilities['chrome']['chromedriverVersion']).split(' ')[0]

#         if browser_version.split('.')[0] != chromedriver_version.split('.')[0]:
#             major_version_different = True

#         print('_________________________________')
#         print('Current web-browser version:\t{}'.format(browser_version))
#         print('Current chrome-driver version:\t{}'.format(chromedriver_version))
#         if major_version_different:
#             print('warning: Version different')
#             print(
#                 'Download correct version at "http://chromedriver.chromium.org/downloads" and place in "./chromedriver"')
#         print('_________________________________')

    def crawl_comment(self, url):
        """Crawl all comment in a post url
        
        {url}: url of a post that you want to crawl
        """
        
        self.browser.get(url)
        #chỉnh lại số trừ theo tốc độ mạng hiện tại lúc chạy
        random_time1 = 9.5 - random.randint(4, 7)
        time.sleep(random_time1)

        #scroll down 
        total_height = int(self.browser.execute_script("return document.body.scrollHeight"))
        for i in range(1, total_height, 1000):
            self.browser.execute_script("window.scrollTo(0, {});".format(i))
            time.sleep(0.3)
        
        full_page_html = self.browser.page_source
        
        try:
            result = self.extract_info(full_page_html)
            self.result.extend(result)
        except Exception as e:
            print("An exception occurred: check internet if needed") 
            logging.error(traceback.format_exc())

    def extract_info(self, html_doc): 
        """Extract 
        [post_name; 
        url;
        reply_number;
        view_number] of each post in one html page"""
        all_cmt = []
        # parse html doc
        soup = BeautifulSoup(html_doc, "html.parser")
        
        cmt = soup.find_all('div', class_ = 'bbWrapper')
        for comment in cmt:
            comment = comment.text
            comment = comment.replace("\n", ' ')
            comment = comment.split()
            comment = " ".join(comment)
            all_cmt.append(comment)
            
        return all_cmt

In [18]:
all_cmt_list = []
no_gui = False

for a_url in tong_hop["url"].values:
    collect = Collect_Xamvn_Info(no_gui=no_gui)
    try:
        collect.crawl_comment("https://" + a_url)
        all_cmt_list.extend(collect.result)
    except:
        print(f"failed: {a_url}")

    collect.browser.quit()


Detected OS : Windows


C:\Users\KIEN\AppData\Local\Temp\ipykernel_16336\88252714.py:36: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  self.browser = webdriver.Chrome(executable_path=executable, chrome_options=chrome_options)
C:\Users\KIEN\AppData\Local\Temp\ipykernel_16336\88252714.py:36: DeprecationWarning: use options instead of chrome_options
  self.browser = webdriver.Chrome(executable_path=executable, chrome_options=chrome_options)


Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected OS : Windows
Detected O

In [21]:
import pandas as pd
df = pd.DataFrame({"comment": all_cmt_list})

In [24]:
df.to_csv("tổng hợp comment.csv")